In [36]:
import pandas as pd
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from time import strftime, strptime, mktime

In [37]:
df = pd.read_csv('./train.csv')
df.shape

(69713, 22)

In [38]:
test = pd.read_csv('./test.csv')
test.shape

(30037, 21)

In [39]:
df1 = df.copy()
test1=test.copy()

In [40]:
y = df.pop('Approved')

In [41]:
train_id = df.pop('ID')
test_id = test.pop('ID')

In [42]:
def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

In [43]:
%%time
df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

CPU times: user 1.92 s, sys: 186 ms, total: 2.11 s
Wall time: 2.1 s


In [44]:
tmp_df = df.copy()
tmp_test = test.copy()

In [45]:
df = tmp_df.copy()
test = tmp_test.copy()

In [47]:
%%time
df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

CPU times: user 50 ms, sys: 1 ms, total: 51 ms
Wall time: 50 ms


In [48]:
%%time
df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

CPU times: user 7.96 s, sys: 2.99 s, total: 11 s
Wall time: 10.9 s


In [49]:
df.shape, test.shape

((69713, 37416), (30037, 37416))

In [50]:
(df.columns == test.columns).sum()

37416

In [109]:
%%time
m = lambda *x: [i.shape for i in x]
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
print m(x_train, y_train, x_test, y_test)

[(55770, 37416), (55770,), (13943, 37416), (13943,)]
CPU times: user 4.6 s, sys: 893 ms, total: 5.5 s
Wall time: 5.49 s


In [110]:
from sklearn.ensemble import RandomForestClassifier


In [157]:
model = RandomForestClassifier(n_estimators=600, min_samples_split = 6, n_jobs=24)
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=24,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [158]:
%%time
model.fit(x_train ,y_train)

CPU times: user 1h 21min 13s, sys: 5.47 s, total: 1h 21min 18s
Wall time: 3min 36s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=24,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [159]:
pred = model.predict(x_test)
predb = model.predict_proba(x_test)

prob = predb[:,1]
roc_auc_score(y_test, prob)

0.8224773007456205

2000 3 82

1200 3 0.8214969566363826

700 2 0.8191452351928523

700 3 0.8246550762532046

500 3 0.8206439658016769

600 4 0.8249777735748512

600 3 0.8223318281027606

600 5 0.8270832441745368

600 9 0.8252744135191886

600 6 0.8224773007456205

In [160]:
f_model = RandomForestClassifier(n_estimators=600, n_jobs=24, min_samples_split=5)
f_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=24,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [161]:
%%time
f_model.fit(df ,y)

CPU times: user 1h 44min 45s, sys: 6.82 s, total: 1h 44min 51s
Wall time: 4min 39s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=24,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [162]:
model = f_model

___

In [163]:
%%time
prob_test = model.predict_proba(test)

CPU times: user 26.3 s, sys: 2.39 s, total: 28.7 s
Wall time: 5.98 s


In [164]:
prob_s = prob_test[:,1]
prob_s

array([0.00055556, 0.03274904, 0.00308333, ..., 0.        , 0.00163889,
       0.04071627])

In [165]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(prob_s, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [166]:
submit.to_csv('fr600.t5_f.csv',index=False)

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba